In [2]:
# import pymysql
import pandas as pd
import numpy as np
import pymysql

# 创建连接
cnxn = pymysql.connect(host='wamvem-dpts.southeastasia.cloudapp.azure.com', port=3306, user='mike', passwd='scott', db='dpts', charset='utf8')
sql = """
SELECT
P1.campaign_name
,P1.brands_name
,P1.startdate
,P1.enddate
,P2.item_id
,P2.cost_item_name
,P2.package_costschedule_purchasetype 
,P2.total_costperiod_volume
,P2.total_costperiod_day
,P2.total_purchasecost
,S1.all_site_name
,P3.placement_id
,P3.placement_name
,P3.location
,P3.format
,P3.creativetype
adservecampaign_id
FROM dpts.sda_campaign P1
inner join dpts.sda_item P2
ON P1.campaign_id = P2.campaign_id
inner join dpts.sda_placement P3
on P2.item_id = P3.item_id
inner join dpts.sda_site S1
on P2.package_site_id = S1.package_site_id
where P1.advertiser_id = '1c5195d0-9e18-47ef-af28-df403b3e77bb' 
"""
# 查询华为荣耀的全部广告位
df_plcm = pd.read_sql(sql,cnxn)
# 去除重复数据
df_plcm = df_plcm.groupby('placement_id').first().reset_index()

df_plcm.head()


,placement_id,campaign_name,brands_name,startdate,enddate,item_id,cost_item_name,package_costschedule_purchasetype,total_costperiod_volume,total_costperiod_day,total_purchasecost,all_site_name,placement_name,location,format,adservecampaign_id
0,0001f0ea-c333-4649-b050-f10dad5f1c3f,华为中国区2016年Q4网络投放,华为企业业务BG中国区(HEBGCN),2016-10-17,2016-12-30,c92631ea-6961-45a0-a55f-1bd1b061962c,None,Fixed,660000.0,12,47000.0,d1net(D1NET)(IT),TE20,D1Net统一通信频道首页B01 Banner位,D1Net统一通信频道首页B01 Banner位,StandardDisplay
1,0004489d-bf57-4ce5-8bbf-6cfc2cf5f247,华为中国区2017年Q1网络投放,华为企业业务BG中国区(HEBGCN),2017-01-05,2017-03-31,732b76fa-55b9-45a6-b648-fb205116132e,敏捷分布式Wi-Fi方案 banner/原有的WLAN广告Banner,Fixed,1120000.0,4,93000.0,51CTO(PC)(51CTO)(IT),敏捷分布式Wi-Fi方案 banner/原有的WLAN广告Banner,51CTO网络无线通栏,51CTO网络无线通栏,StandardDisplay
2,0006cb31-4f92-4e92-83bc-81cdd91eba41,集团支持Mate 7投放项目-12月,华为集团业务BG机关(HCORP),2014-12-01,2014-12-31,711455b5-f41b-4b7b-878c-f8bec99469a8,新浪网(SINA)_新浪网_科技频道_科技首页全屏_1000x45_CPH：13:0016,BonusC,24.0,6,342000.0,新浪门户(PC)(SINA)(IT),新浪网(SINA)_新浪网_科技频道_科技首页全屏_1000x45_CPH：13:0016,科技频道,科技首页全屏,StandardDisplay
3,000859c7-a077-4a59-92e5-20a1c799baa4,荣耀6plus9-10月项目-网易（MO）,荣耀(Honor),2015-10-12,2015-11-05,e69f60b9-224d-487d-a663-074ed0fa05ad,None,Fixed,0.0,0,0.0,网易163门户(PC)(163)(IT),北京广易通广告有限公_网易_易信官方公众号_要闻区第二条图文，图片体现客户_1x1,要闻区第二条图文，图片体现客户LOGO,,1.0
4,00087be6-1114-4ce0-9629-bab0db0b12a4,荣耀K推广投放——门户,荣耀(Honor),2016-04-29,2016-07-08,3a70b5c3-5ced-46ee-8d95-5cc6d77e2540,None,Fixed,0.0,56,396000.0,网易163门户(PC)(163)(IT),BJWYCMYXGS_网易163门户(PC_新闻频道文章内页_触发式擎天柱广告（12轮替）_1x1,新闻频道-文章内页,触发式擎天柱广告（1/2轮替）,StandardDisplay


query cost period data to dataframe

In [5]:
def split_site(df):
    for key, val in df.all_site_name.iteritems():
        v = val.split('(')
        m_ = v[0]
        if 'ChinaByte' in df.loc[key,'placement_name'] or '比特网' in df.loc[key,'placement_name']:
            m_ = '比特网'
        elif 'None' in m_ and  len(df.loc[key,'placement_name'].split('_'))>1:
            m_ = df.loc[key,'placement_name'].split('_')[1]
#             m_ = re.findall('([^\(\)].)+',m_)[0]
        
        df.loc[key,'media'] = m_
df_plcm.pipe(split_site)
# df.all_site_name
# df_plcm.all_site_name.iteritems

In [6]:
df_plcm[df_plcm.media.str.contains('None')].head()

,placement_id,campaign_name,brands_name,startdate,enddate,item_id,cost_item_name,package_costschedule_purchasetype,total_costperiod_volume,total_costperiod_day,total_purchasecost,all_site_name,placement_name,location,format,adservecampaign_id,media
344,05b3eb13-6506-467e-a180-03ee6a4639b6,华为中国区2016年Q3网络投放,华为企业业务BG中国区(HEBGCN),2016-07-13,2016-10-21,eff9a8f3-d432-42c1-9127-b95b3c321c3d,None,,720000.0,2,30000.0,IT Technology|||Other|||None,敏捷网络 banner & 数据中心 banner,比特网网络频道首页通栏,比特网网络频道首页通栏,StandardDisplay,IT Technology|||Other|||None
1233,15eab5f4-93ec-42d5-a948-1f18d501a510,华为云服务百度关键字搜索2016年第八波、360及搜狗投放,华为企业业务BG中国区(HEBGCN),2016-06-08,2016-06-30,48ad63c7-4760-47a3-99e5-f3fa644dd841,None,,72000.0,15,15000.0,IT Technology|||Search and directories|||None,华为企业云服务业务,网页,搜索关键字,SearchKeyword,IT Technology|||Search and directories|||None
2198,2791bd07-d9a4-46c0-88e5-b2037269b4f8,华为中国区2016年Q3网络投放,华为企业业务BG中国区(HEBGCN),2016-07-13,2016-10-21,d6deff35-4577-41f0-b6d7-62c9e7e05e03,None,,720000.0,2,0.0,IT Technology|||Other|||None|||Lifestyle and C...,智慧城市形象广告,比特网网络频道首页通栏,比特网网络频道首页通栏,StandardDisplay,IT Technology|||Other|||None|||Lifestyle and C...
3514,3ea0d641-aef0-434d-9d37-f08768dc5d0d,华为云服务搜狗搜索2016年第三波投放,华为企业业务BG中国区(HEBGCN),2016-05-30,2016-06-13,039edfa9-7eb2-468b-9b24-1b1dbbd7674e,None,,72000.0,15,15000.0,IT Technology|||Search and directories|||None,华为企业云服务业务,网页,搜索关键字,SearchKeyword,IT Technology|||Search and directories|||None
4256,4c597d3f-af78-46fd-93db-8406fd8693e2,华为中国区2016年Q2网络投放,华为企业业务BG中国区(HEBGCN),2016-04-12,2016-07-10,55413aef-56a7-44de-95e2-74b4231eb9d1,None,,1620000.0,4,65000.0,IT Technology|||None,S2600T,比特存储频道首页通栏,比特存储频道首页通栏,StandardDisplay,IT Technology|||None


In [8]:
cnxn.connect()
sql = """
SELECT 
`item_id`,
P2.date,1 as days
FROM `sda_costperiod` P1
inner join `elt_date` P2
on P2.date BETWEEN P1.package_costschedule_costperiod_startdate and P1.package_costschedule_costperiod_enddate
where advertiser_id = '1c5195d0-9e18-47ef-af28-df403b3e77bb' 
"""
df_spot = pd.read_sql(sql,cnxn)

import chinese_calendar as calendar

def convert_date(df):
    """
    convert date to datetime type
    add feature for holiday
    """
    df['date'] = df.date.astype(pd.datetime)
    
    return df

df_spot = df_spot.pipe(convert_date)

day_holiday = {}
day_holiday_name = {}
for day in df_spot.date.unique():
    on_holiday, holiday_name = calendar.get_holiday_detail(day)
    day_holiday[day] = on_holiday 
    day_holiday_name[day] = holiday_name
df_spot['is_holiday'] = df_spot.date.map(day_holiday)
df_spot['holiday_name'] = df_spot.date.map(day_holiday_name).fillna('')
df_spot.head()

,item_id,date,days,is_holiday,holiday_name
0,333ab8e2-f467-4fc8-9a92-18349403147d,2016-01-04,1,False,
1,333ab8e2-f467-4fc8-9a92-18349403147d,2016-01-07,1,False,
2,333ab8e2-f467-4fc8-9a92-18349403147d,2016-01-11,1,False,
3,333ab8e2-f467-4fc8-9a92-18349403147d,2016-01-12,1,False,
4,333ab8e2-f467-4fc8-9a92-18349403147d,2016-01-13,1,False,


读取Admaster数据 (荣耀brand:20435 advertiser:10040, network: 48)
drop table  if EXISTS`raw_admaster_dailyreport`;

create table raw_admaster_dailyreport
(
am_network_id	INT	
,am_campaign_id	INT	
,am_network_media_id	INT
,am_placement_id	INT	
,`date` date
,imp	INT	
,clk	INT	
,uimp	INT
,uclk	INT
)
delete from raw_admaster_dailyreport;
INSERT into raw_admaster_dailyreport
SELECT   
`am_network_id`,`am_campaign_id`, `am_network_media_id`, `am_placement_id`,
DATE_FORMAT(`create_datetime`,'%Y-%m-%d') as `date`,
sum(imp) imp , sum(clk) clk, sum(`uimp`) uimp, sum(`uclk`) uclk
FROM `raw_admaster_region`
where `am_network_id`=48 and `create_datetime`>='2017-01-01'
GROUP BY  `project_id`,`am_network_id`,`am_campaign_id`, `am_network_media_id`, `am_placement_id`,DATE_FORMAT(`create_datetime`,'%Y-%m-%d')

In [51]:

cnxn.connect()
sql = """
SELECT
P1.`am_campaign_id`,P1.`campaign_name`, P1.`default_target`,
`placement_name`, P2.`real_imp`, P2.`real_clk`,
P3.date,P3.imp, P3.uimp, P3.clk, P3.uclk
FROM `raw_admaster_campaign` P1
inner join raw_admaster_placement P2
on P1.am_campaign_id =  P2.am_campaign_id
inner join raw_admaster_dailyreport P3
on  P2.am_network_id=P3.am_network_id and P2.am_campaign_id=P3.am_campaign_id and P2.am_placement_id=P3.am_placement_id
WHERE `am_brand_id`=20435 and (P3.imp>1000 or P3.clk>10)

"""
df_perf = pd.read_sql(sql,cnxn)
df_perf.head()

,am_campaign_id,campaign_name,default_target,placement_name,real_imp,real_clk,date,imp,uimp,clk,uclk
0,63682,荣耀2016年固定位年框合作（移动）项目,http://msale.vmall.com/honor.html?cid=63402,闪屏广告_静态可外链（轮播）,9,0,2017-01-01,0,0,52,50
1,63682,荣耀2016年固定位年框合作（移动）项目,http://msale.vmall.com/honor.html?cid=63402,闪屏广告_静态可外链（轮播）,9,0,2017-01-10,0,0,23,23
2,63682,荣耀2016年固定位年框合作（移动）项目,http://msale.vmall.com/honor.html?cid=63402,闪屏广告_静态可外链（轮播）,9,0,2017-01-11,0,0,53,53
3,63682,荣耀2016年固定位年框合作（移动）项目,http://msale.vmall.com/honor.html?cid=63402,闪屏广告_静态可外链（轮播）,9,0,2017-01-15,0,0,65,65
4,63682,荣耀2016年固定位年框合作（移动）项目,http://msale.vmall.com/honor.html?cid=63402,闪屏广告_静态可外链（轮播）,9,0,2017-01-20,0,0,11,11


In [ ]:
df_plcm.merge(df_perf, left_on=['adservecampaign_id','placement_name'], right_on=['am_campaign_id','placement_name'])